In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available, check CUDA and cuDNN installation")


2024-06-24 11:38:42.123950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 11:38:42.852751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0
GPU is not available, check CUDA and cuDNN installation


2024-06-24 11:38:43.912336: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

2024-06-24 11:20:24.822569: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 11:20:25.648350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [29]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[]

In [23]:
X = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/academic_success/train.csv')
y = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/academic_success/test.csv')

In [24]:
def preprocess(data):
    dict = {
        'Enrolled': 1,
        'Dropout': 0,
        'Graduate': 2,
    }
    data['Target'] = data['Target'].map(dict)
    return data
X = preprocess(X)
X

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,0
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,1
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,2
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,2
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,1
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,0


In [25]:
def new_features(data):
    data['Total_Curricular_units_enrolled'] = data['Curricular units 1st sem (enrolled)'] + data['Curricular units 2nd sem (enrolled)']
    data['Total_Curricular_units_approved'] = data['Curricular units 1st sem (approved)'] + data['Curricular units 2nd sem (approved)']
    data['Total_Curricular_units_grade'] = data['Curricular units 1st sem (grade)'] + data['Curricular units 2nd sem (grade)']
    data['Average_Curricular_units_grade'] = data['Total_Curricular_units_grade'] / (data['Total_Curricular_units_enrolled'] + 1e-9)
    data['Performance_ratio'] = data['Total_Curricular_units_approved'] / (data['Total_Curricular_units_enrolled'] + 1e-9)
    data['tuition_debtor'] = ((data['Tuition fees up to date'] == 0) & (data['Debtor'] == 1)).astype(int)

    data['Academic_growth'] = data['Curricular units 2nd sem (grade)'] - data['Curricular units 1st sem (grade)']
    data['Stability_index'] = abs(data['Curricular units 1st sem (enrolled)'] - data['Curricular units 2nd sem (enrolled)']) + abs(data['Curricular units 1st sem (approved)'] - data['Curricular units 2nd sem (approved)'])
    data['Financial_strain'] = ((data['Scholarship holder'] == 0) & (data['tuition_debtor'] == 1)).astype(int)

    return data

X = new_features(X)

In [26]:
y = X['Target']
X = X.drop(['id', 'Target'], axis=1)

In [27]:

encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(y)
# Convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Neural network architecture
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/50


/opt/conda/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-24 11:17:07.652278: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1531/1531 ━━━━━━━━━━━━━━━━━━━━ 2s 795us/step - accuracy: 0.6966 - loss: 0.7652 - val_accuracy: 0.8066 - val_loss: 0.4921
Epoch 2/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - accuracy: 0.7961 - loss: 0.5487 - val_accuracy: 0.8113 - val_loss: 0.4778
Epoch 3/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 0.8034 - loss: 0.5257 - val_accuracy: 0.8117 - val_loss: 0.4716
Epoch 4/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 0.8068 - loss: 0.5180 - val_accuracy: 0.8174 - val_loss: 0.4699
Epoch 5/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 1s 746us/step - accuracy: 0.8060 - loss: 0.5156 - val_accuracy: 0.8157 - val_loss: 0.4690
Epoch 6/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.8106 - loss: 0.5084 - val_accuracy: 0.8170 - val_loss: 0.4654
Epoch 7/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 2s 984us/step - accuracy: 0.8112 - loss: 0.5080 - val_accuracy: 0.8173 - val_loss: 0.4707
Epoch 8/50
1531/1531 ━━━━━━━━━━━━━━━━━━━━ 2s 963us/step - accuracy: 0.8084 - loss: 0.50

KeyboardInterrupt: 